In [1]:
'''
Clayton Cohn
20200228
CSC440
Assignment 5

75 points

'''

'''
1. (50 points) Meet in the middle attack on 2DES Implement the meet-in-the-middle attack on 2DES, 
where you use the simplified DES from the textbook. Follow the instructions on pp. 143-144. 
Because you're using the simplifed DES, which has a key containing 9 bits, you will need to generate two lists, 
each of length 512.

You should have the DES encryption function implemented in assignment 4 and you will now need to implement the 
decryption function. Write the program so that it outputs a list of possible key pairs. Here are two sets of inputs 
on which to test your program:

Plaintext: 101, ciphertext: 199
Plaintext: 110, ciphertext: 2816

Each of those inputs will produce a list of about 50 possible key pairs Write your program so that constructs lists 
of key pairs for each input and then finds and prints the key pair(s) that appear on both lists.
'''

# Using my encrypt method from Assignment 4

import binascii
from textwrap import wrap

def encrypt(plaintext, key):
    
    plaintext = bin(plaintext)[2:]
    while (len(plaintext) < 12): plaintext = "0" + plaintext

    key = bin(key)[2:]
    while (len(key) < 9): key = "0" + key

    print("\nPlaintext: {0} ({1})".format(plaintext, int(plaintext,2)))
    print("\nKey: {0} ({1})".format(key,int(key,2)))
    
    # First make sure arguments are without bounds
    if len(plaintext) not in range(0, 4096) or len(key) not in range(0, 512):
        print("Invalid lengths.")
        return
    
    # Identify the s-boxes
    s1 =    [
                ["101", "010", "001", "110", "011", "100", "111", "000"],
                ["001", "100", "110", "010", "000", "111", "101", "011"] 
            ]
    s2 =    [
                ["100", "000", "110", "101", "111", "001", "011", "010"],
                ["101", "011", "000", "111", "110", "010", "001", "100"]
            ]
    
    # Separate plaintext into 12-bit blocks
    blocks = wrap(plaintext,12)
    
    # Pad last block with 0s
    blocks[-1] = blocks[-1].ljust(12,"0")[:12]
    
    # Generate subkeys from key
    long_key = key * 2
    subkeys = [long_key[0:8], long_key[1:9], long_key[2:10], long_key[3:11]]
    print("\nSubkeys: {}".format(subkeys))
    
    print("\nBlocks: {0}".format(blocks))
    
    # Create ciphertext
    ciphertext = ""
    
    # For each block...
    for i in range(0,len(blocks)):
        block = blocks[i]
        
        # Split block into left and right
        l, r = block[:len(block)//2], block[len(block)//2:]
        print("\nBlock {0}\nLeft: {1} ({2})\nRight: {3} ({4})".format(i,l,int(l,2),r,int(r,2)))
        
        # 4 rounds for simplified version
        for j in range (0,4):
            
            # Copy previous left and right
            l_old = l
            
            # Get appropriate key
            k = subkeys[j]
            
            # Assign next l to current r
            l = r
            
            # Perform f on r
        
            # First expand the r
            r = expand(r)
            
            # XOR k (keys[i]) with r
            r = xor(r,k,8)
            
            # Split bits and prepare for s-boxes
            r1, r2 = r[:len(r)//2], r[len(r)//2:] 
            print("r1: " + r1)
            print("r2: " + r2)
            
            s1row = int(r1[0])
            s1col = int(r1[1:],2)
            print("S1 row: {0}, S1 col: {1}".format(s1row,s1col))
    
            s2row = int(r2[0])
            s2col = int(r2[1:],2)
            print("S2 row: {0}, S2 col: {1}".format(s2row,s2col))
            
            sbox1_choice = s1[s1row][s1col]
            sbox2_choice = s2[s2row][s2col]
            print("S1 choice: {0}, S2 choice: {1}".format(sbox1_choice,sbox2_choice))
            
            #Output of function f is those 6 bits
            f = sbox1_choice + sbox2_choice
            
            # To get the new r, we need to xor f with the previous l
            r = xor(l_old,f,6)
            print("L: {0}, R: {1}, ".format(l,r,int(subkeys[j],2)))
            print("END Block {0} - Round {1}: L = {2} ({3}), R = {4} ({5}), Subkey = {6} ({7})" \
                  .format(i,j,l,int(l,2),r,int(r,2),subkeys[j],int(subkeys[j],2)))
            
            print("\n")
            
        # Add blocks to ciphertext
        ciphertext += l + r
    
    print("Final ciphertext: " + ciphertext)
    if len(ciphertext) == 0: return 0
    else: ciphertext = int(ciphertext, 2)
        
    # Return int ciphertext
    # print("\nCiphertext: {0}".format(ciphertext))
    return ciphertext
        
# Helper function to use as E for expansion from 6 bits to 8
def expand(s):
    result = s[0] + s[1] + s[3] + s[2] + s[3] + s[2] + s[4] + s[5]
    print(s + " expanded to " + result)
    return result

# Helper function to xor two bit strings
def xor(s1, s2, n):
    n1 = int(s1,2)
    n2 = int(s2,2)
    result = bin(n1^n2)[2:]
    
    # If result is not n bits, we must 0 append to beginning of result
    nZeros = n-len(result)
    zs = "0" * nZeros
    result = zs + result
    
    print(s1 + " XOR " +  s2 + " = " + result)
    return result

In [3]:
# Run the encryption
pt = input("Please enter plaintext: ")
k = input("Please enter key: ")
if (pt.isdigit() and k.isdigit()): 
    n = encrypt(int(pt), int(k))
    print("\nReturn value: " + str(n))
else: 
    print("Either your plaintext or key was not a deimal number. Please try again .")

Please enter plaintext: 1830
Please enter key: 203

Plaintext: 011100100110 (1830)

Key: 011001011 (203)

Subkeys: ['01100101', '11001011', '10010110', '00101101']

Blocks: ['011100100110']

Block 0
Left: 011100 (28)
Right: 100110 (38)
100110 expanded to 10101010
10101010 XOR 01100101 = 11001111
r1: 1100
r2: 1111
S1 row: 1, S1 col: 4
S2 row: 1, S2 col: 7
S1 choice: 000, S2 choice: 100
011100 XOR 000100 = 011000
L: 100110, R: 011000, 
END Block 0 - Round 0: L = 100110 (38), R = 011000 (24), Subkey = 01100101 (101)


011000 expanded to 01010100
01010100 XOR 11001011 = 10011111
r1: 1001
r2: 1111
S1 row: 1, S1 col: 1
S2 row: 1, S2 col: 7
S1 choice: 100, S2 choice: 100
100110 XOR 100100 = 000010
L: 011000, R: 000010, 
END Block 0 - Round 1: L = 011000 (24), R = 000010 (2), Subkey = 11001011 (203)


000010 expanded to 00000010
00000010 XOR 10010110 = 10010100
r1: 1001
r2: 0100
S1 row: 1, S1 col: 1
S2 row: 0, S2 col: 4
S1 choice: 100, S2 choice: 111
011000 XOR 100111 = 111111
L: 000010, R: 11

In [ ]:
'''
2. (25 points) Baby step, giant step attack on discrete log Implement the attack described in section 7.2.2 of 
the textbook. You will need these values:

p = 595117
alpha = 1002
alpha^x = 437083

Your program will output x, which will be an integer where each pair of digits repressents a letter in the encoding 
scheme A = 01, B = 02, ..., Z = 26. Please translate the integer to an acronym.
'''

